<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item nDCG </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [5]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [11]:
def model_trainer(neighbours):
    neighbours=int(neighbours)
    print(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    return -results.groupby('Algorithm').ndcg.mean()


In [14]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': model_trainer(x), 'status': STATUS_OK }

average=[[] for _ in range(10)]

for y in range(len(average)):
    best = fmin(objective,
        space=hp.uniform('x', 1, 50),
        algo=tpe.suggest,
        max_evals=10
               )
    average[y].append(best)

9                                                                                                                      
12                                                                                                                     
41                                                                                                                     
46                                                                                                                     
6                                                                                                                      
47                                                                                                                     
36                                                                                                                     
11                                                                                                                     
9                                       

In [34]:
average =[[{'x': 46.21239562832436}],
 [{'x': 11.18767077609165}],
 [{'x': 19.472446054398755}],
 [{'x': 29.40258057625774}],
 [{'x': 17.557133113273665}],
 [{'x': 10.565668614323931}],
 [{'x': 20.53518511564948}],
 [{'x': 10.658761442128954}],
 [{'x': 11.237982203247403}],
 [{'x': 6.7039056407556625}]]

In [ ]:
flat_list = [item for sublist in average for item in sublist]

In [29]:
ndcg = [0.10078672601755276,0.10307457304854108,0.1027919187220432,0.1025849982138326,0.10031851123659034,0.10245817130009784,
0.1020391138278402,0.10304630476630926,0.10159226363831887,0.10068421182413598]
ndcg = pd.DataFrame(ndcg)

In [37]:
average = pd.DataFrame.from_dict(flat_list)
metric = pd.concat([average,ndcg],axis=1)
metric.columns = ['Neighbours','nDCG']
metric

,Neighbours,nDCG
0,46.212396,0.100787
1,11.187671,0.103075
2,19.472446,0.102792
3,29.402581,0.102585
4,17.557133,0.100319
5,10.565669,0.102458
6,20.535185,0.102039
7,10.658761,0.103046
8,11.237982,0.101592
9,6.703906,0.100684


In [39]:
metric.mean()

Neighbours    18.353373
nDCG           0.101938
dtype: float64